In [1]:
import torch

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, _ = tokenizer.encode('how are you', max_length=6)

input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

input_ids, attention_mask, tokenizer.decode(input_ids)

/root/miniconda3/envs/cuda117/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(tensor([   1,    1,    0, 9178,   32,   47]),
 tensor([0, 0, 1, 1, 1, 1]),
 '<pad><pad><s>how are you')

In [2]:
from datasets import load_dataset
from transformers import default_data_collator

dataset = load_dataset(
    'json',
    data_files='/root/code/DeepSpeed-Chat_my/data/train.json',
    split='train')

#2,4,4切分,取最后一部分
dataset = dataset.select(range(45000, len(dataset)))


def f(data):
    input_ids, _ = tokenizer.encode(data['prompt'], max_length=256)
    input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

    return {'input_ids': input_ids, 'attention_mask': attention_mask}


dataset = dataset.map(f, remove_columns=dataset.column_names)

loader = torch.utils.data.DataLoader(dataset,
                                     collate_fn=default_data_collator,
                                     batch_size=4,
                                     shuffle=True,
                                     drop_last=True)

len(loader), next(iter(loader))

(7144,
 {'input_ids': tensor([[   1,    1,    1,  ...,  116, 6267,   35],
          [   1,    1,    1,  ...,  116, 6267,   35],
          [   1,    1,    1,  ...,  116, 6267,   35],
          [   1,    1,    1,  ...,  116, 6267,   35]]),
  'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
          [0, 0, 0,  ..., 1, 1, 1],
          [0, 0, 0,  ..., 1, 1, 1],
          [0, 0, 0,  ..., 1, 1, 1]])})

In [3]:
from transformers import AutoConfig, AutoModelForCausalLM, get_scheduler
import lora

model_actor = AutoModelForCausalLM.from_pretrained('model/actor', dropout=0.0)
lora.insert(model_actor)


def f():
    params = []
    params_lora = []
    for n, p in model_actor.named_parameters():
        if not p.requires_grad:
            continue
        if 'lora_A' in n or 'lora_B' in n:
            params_lora.append(p)
            continue
        params.append(p)

    return [{
        'params': params,
        'weight_decay': 0.0
    }, {
        'params': params_lora,
        'weight_decay': 0.0,
        'lr': 5e-4
    }]


optimizer_actor = torch.optim.Adam(f(), lr=1e-5, betas=(0.9, 0.95))

scheduler_actor = get_scheduler(name='cosine',
                                optimizer=optimizer_actor,
                                num_warmup_steps=100,
                                num_training_steps=800)

model_actor.gradient_checkpointing_enable()
model_actor.train()

lora.count_params(model_actor)

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.13s/it]


{'count_require': 2.21044736, 'count_all': 14.29004288, 'ratio': 0.15468444556549854}


In [4]:
class CriticModel(torch.nn.Module):

    def __init__(self):
        super().__init__()

        from transformers import AutoModel
        self.rwtransformer = AutoModel.from_pretrained('facebook/opt-350m',
                                                       dropout=0.0)

        self.v_head = torch.nn.Linear(512, 1, bias=False)

    def get_value(self, input_ids, attention_mask):
        value = self.rwtransformer(
            input_ids=input_ids,
            attention_mask=attention_mask).last_hidden_state
        return self.v_head(value).squeeze(2)

    def get_reward(self, input_ids, attention_mask):
        value = self.get_value(input_ids, attention_mask)

        reward = []
        for i, v in zip(input_ids, value):
            end = input_ids.shape[1] - 1
            if tokenizer.eos_token_id in i:
                end = i.tolist().index(tokenizer.eos_token_id)
            reward.append(v[end])
        reward = torch.stack(reward)

        return reward


model_critic = torch.load('model/critic')

optimizer_critic = torch.optim.Adam(model_critic.parameters(),
                                    lr=5e-6,
                                    betas=(0.9, 0.95))

scheduler_critic = get_scheduler(name='cosine',
                                 optimizer=optimizer_critic,
                                 num_warmup_steps=100,
                                 num_training_steps=800)

model_critic.train()

lora.count_params(model_critic)

{'count_require': 3.31196928, 'count_all': 3.31196928, 'ratio': 1.0}


In [5]:
from accelerate import Accelerator

model_ref = AutoModelForCausalLM.from_pretrained('model/actor')
model_reward = torch.load('model/critic')

model_ref.eval()
model_reward.eval()

accelerator = Accelerator(gradient_accumulation_steps=1,
                          mixed_precision='fp16')

(loader, model_actor, optimizer_actor, scheduler_actor, model_critic,
 optimizer_critic,
 scheduler_critic) = accelerator.prepare(loader, model_actor, optimizer_actor,
                                         scheduler_actor, model_critic,
                                         optimizer_critic, scheduler_critic)

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.08s/it]
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [6]:
@torch.no_grad()
def get_generate(input_ids, attention_mask):
    generate = model_actor.generate(input_ids,
                                    attention_mask=attention_mask,
                                    max_length=512,
                                    pad_token_id=tokenizer.pad_token_id,
                                    eos_token_id=tokenizer.eos_token_id)

    lens = (generate[:, 256:] != tokenizer.pad_token_id).sum(1)

    return generate[lens > 1]


data = next(iter(loader))

get_generate(**data).shape

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/miniconda3/envs/cuda117/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


torch.Size([4, 299])

In [7]:
def get_prob(prob, index):
    prob = prob.log_softmax(dim=2)
    prob = prob.gather(dim=2, index=index.unsqueeze(2))
    return prob.squeeze(2)


get_prob(torch.randn(4, 123, 999), torch.randint(0, 999, (4, 123))).shape

torch.Size([4, 123])

In [8]:
last_generate = None


@torch.no_grad()
def get_batch(input_ids, attention_mask):
    #input_ids -> [4, 256]
    #attention_mask -> [4, 256]
    global last_generate

    #根据问题生成回答
    #[4, gen_lens]
    generate = get_generate(input_ids, attention_mask)

    #制作缓存,防止所有回答为空的情况
    if len(generate):
        last_generate = generate
    else:
        generate = last_generate

    #[4, gen_lens]
    generate_mask = (generate != tokenizer.pad_token_id).long()

    #两个模型分别取回答被预测到的概率
    #[4, gen_lens-1]
    prob_old = model_actor(input_ids=generate,
                           attention_mask=generate_mask).logits
    prob_old = get_prob(prob_old[:, :-1], generate[:, 1:])

    #取每个词的value
    #[4, gen_lens-1]
    value_old = model_critic.get_value(generate, generate_mask)[:, :-1]

    #[4, gen_lens-1]
    prob_ref = model_ref(
        input_ids=generate.to('cpu'),
        attention_mask=generate_mask.to('cpu')).logits.to('cuda')
    prob_ref = get_prob(prob_ref[:, :-1], generate[:, 1:])

    #取回答的分数
    #[4]
    reward = model_reward.get_reward(generate.to('cpu'),
                                     generate_mask.to('cpu')).to('cuda')

    return generate, generate_mask, prob_old, prob_ref, value_old, reward


generate, generate_mask, prob_old, prob_ref, value_old, reward = get_batch(
    **data)

generate.shape, generate_mask.shape, prob_old.shape, prob_ref.shape, value_old.shape, reward.shape

(torch.Size([4, 299]),
 torch.Size([4, 299]),
 torch.Size([4, 298]),
 torch.Size([4, 298]),
 torch.Size([4, 298]),
 torch.Size([4]))

In [9]:
def get_reward_kl(end, prob_old, prob_ref, reward):
    #prob_old -> [4, gen_lens-1]
    #prob_ref -> [4, gen_lens-1]
    #reward -> [4]

    #两份预测概率求kl散度
    #[4, gen_lens-1]
    reward_kl = -0.1 * (prob_old - prob_ref)

    #把原本的reward加在kl散度的最后一个字上
    for i, e in enumerate(end):
        if e >= reward_kl.shape[1]:
            e = -1
        reward_kl[i, e] += reward[i].clamp(-5, 5)

    #[4, gen_lens-1]
    return reward_kl


end = generate_mask[:, 256:].sum(1) + 255
end = end.tolist()

reward_kl = get_reward_kl(end, prob_old, prob_ref, reward)

reward_kl.shape

torch.Size([4, 298])

In [10]:
#解释见get_delta_note函数
def get_delta(value_old, reward_kl):
    #value_old -> [4, gen_lens-1]
    #reward_kl -> [4, gen_lens-1]

    #gen_lens-2 -> 255
    delta = []
    for i in reversed(range(255, value_old.shape[1])):
        #[4]
        value_next = 0.0
        if i != value_old.shape[1] - 1:
            value_next = value_old[:, i + 1]

        #[4]
        d = reward_kl[:, i] + value_next - value_old[:, i]
        if len(delta):
            d += 0.95 * delta[-1]
        delta.append(d)

    #[4, gen_lens-256]
    delta = torch.stack(delta[::-1], dim=1)

    return delta


delta = get_delta(value_old, reward_kl)

delta.shape

torch.Size([4, 43])

In [11]:
#get_delta函数的原理解释,注释性代码
#数学上和get_delta函数等价,但是运行效率低
def get_delta_note(value_old, reward_kl):
    #循环中自减会出问题,所以先clone一份再操作
    clone = value_old.clone()

    #下一个词的value,减去当前词的value,相当于对value去基线,缩小数值方差
    #每个词的value是相互独立的,前后词value的差,可以视为预测质量的衡量
    for i in range(255, value_old.shape[1]):
        value_next = 0.0
        if i != value_old.shape[1] - 1:
            value_next = value_old[:, i + 1]
        clone[:, i] = value_next - value_old[:, i]
    value_old = clone

    #在value中融合reward,kl
    value_old += reward_kl

    #蒙特卡洛采样法估计Q函数
    #这里计算的其实就是adv
    delta = []
    for i in range(255, value_old.shape[1]):
        s = 0
        for j in range(i, value_old.shape[1]):
            s += value_old[:, j] * 0.95**(j - i)
        delta.append(s)

    return torch.stack(delta, dim=1)


#测试两个函数是等价的,误差是由于计算机精度导致的
for i in range(1000):
    value_old_test = torch.randn(4, 285)
    reward_kl_test = torch.randn(4, 285)

    diff = get_delta(value_old_test, reward_kl_test) - get_delta_note(
        value_old_test, reward_kl_test)
    diff = diff.abs().max().item()
    assert diff < 1e-5
'test success'

'test success'

In [12]:
def get_loss_actor(prob_new, prob_old, delta, generate_mask):
    prob_new = prob_new[:, 255:]
    prob_old = prob_old[:, 255:]
    generate_mask = generate_mask[:, 256:]

    #prob_new -> [4, gen_lens-256]
    #prob_old -> [4, gen_lens-256]
    #delta -> [4, gen_lens-256]
    #generate_mask -> [4, gen_lens-256]

    #对数概率,求差就是求商,所以这里求的是新旧概率的变化率
    #[4, gen_lens-256]
    ratio = ((prob_new - prob_old) * generate_mask).exp()

    #delta是估计出来的去基线Q值,以变化率来缩放Q值
    #最大化Q值,以此来寻找最优的actor
    #裁剪,防止自举
    #[4, gen_lens-256]
    loss1 = delta * ratio
    loss2 = delta * ratio.clamp(0.8, 1.2)
    loss = torch.min(loss1, loss2) * generate_mask
    loss = loss.sum() / generate_mask.sum() / 8
    return -loss


loss_actor = get_loss_actor(prob_old, prob_old, delta, generate_mask)

loss_actor

tensor(0.2290, device='cuda:0')

In [13]:
def get_loss_critic(value_new, value_old, delta, generate_mask):
    value_new = value_new[:, 255:]
    value_old = value_old[:, 255:]
    generate_mask = generate_mask[:, 256:]

    #value_new -> [4, gen_lens-256]
    #value_old -> [4, gen_lens-256]
    #delta -> [4, gen_lens-256]
    #generate_mask -> [4, gen_lens-256]

    #delta是估计出来的去基线Q值,加上value_old后还原为Q值
    #value_new和Q值求mse loss即可,因为value都是对Q函数的估计
    #裁剪,防止自举
    #[4, gen_lens-256]
    loss1 = (value_new - delta - value_old)**2
    value_new = value_new.clamp(value_old - 0.2, value_old + 0.2)
    loss2 = (value_new - delta - value_old)**2

    #求平均
    loss = torch.max(loss1, loss2) * generate_mask
    loss = loss.sum() / 2 / generate_mask.sum() / 8

    return loss


loss_critic = get_loss_critic(value_old, value_old, delta, generate_mask)

loss_critic

tensor(0.4388, device='cuda:0')

In [14]:
def train(generate, generate_mask, prob_old, prob_ref, value_old, reward,
          do_step):
    #generate -> [4, gen_lens]
    #generate_mask -> [4, gen_lens]
    #prob_old -> [4, gen_lens-1]
    #prob_ref -> [4, gen_lens-1]
    #value_old -> [4, gen_lens-1]
    #reward -> [4]
    #do_step -> bool

    #求出每句话结束的索引
    #[4]
    end = generate_mask[:, 256:].sum(1) + 255
    end = end.tolist()

    #结束以后的value归零
    for i, e in enumerate(end):
        value_old[i, e + 1:] = 0

    with torch.no_grad():
        #计算新旧概率的kl散度,再把reward加在最后一个字上
        #[4, gen_lens-1]
        reward_kl = get_reward_kl(end, prob_old, prob_ref, reward)

        #估计去基线的Q值
        #[4, gen_lens-256]
        delta = get_delta(value_old, reward_kl)

    #重新计算回答被生成的概率
    #[4, gen_lens-1]
    prob_new = model_actor(input_ids=generate,
                           attention_mask=generate_mask).logits
    prob_new = get_prob(prob_new[:, :-1], generate[:, 1:])

    #更新actor
    loss_actor = get_loss_actor(prob_new, prob_old, delta, generate_mask)
    accelerator.backward(loss_actor)
    if do_step:
        accelerator.clip_grad_norm_(
            [i for i in model_actor.parameters() if i.requires_grad], 1.0)
        optimizer_actor.step()
        scheduler_actor.step()
        optimizer_actor.zero_grad()

    #重新计算每个词的value
    #[4, gen_lens-1]
    value_new = model_critic.get_value(input_ids=generate,
                                       attention_mask=generate_mask)[:, :-1]

    #更新critic
    loss_critic = get_loss_critic(value_new, value_old, delta, generate_mask)
    accelerator.backward(loss_critic)
    if do_step:
        accelerator.clip_grad_norm_(model_critic.parameters(), 1.0)
        optimizer_critic.step()
        scheduler_critic.step()
        optimizer_critic.zero_grad()

    return loss_actor.item(), loss_critic.item()


train(generate, generate_mask, prob_old, prob_ref, value_old, reward, True)

(0.05860239639878273, 0.14592543244361877)

In [15]:
for i, data in enumerate(loader):
    #生成数据
    (generate, generate_mask, prob_old, prob_ref, value_old,
     reward) = get_batch(**data)

    do_step = (i + 1) % 8 == 0

    #训练
    loss_actor, loss_critic = train(generate, generate_mask, prob_old,
                                    prob_ref, value_old, reward, do_step)

    if do_step:
        lr_actor = optimizer_actor.param_groups[0]['lr']
        lr_critic = optimizer_critic.param_groups[0]['lr']
        print(i, len(loader), loss_actor, loss_critic, lr_actor, lr_critic)
        print(tokenizer.decode(generate[0, 256:]))
        print(reward[0].item())

    if i == 2500:
        break

lora.merge(model_actor)
model_actor.save_pretrained('model/rlhf')

7 7144 0.06728362292051315 0.13812726736068726 2.0000000000000002e-07 1.0000000000000001e-07
SELECT FATHERS_NAME from TABLE_NAME_89 where SPOUSER = "OTTOKAR II" and BIRTH = "1204"</s>
-4.665192604064941
15 7144 0.05079767480492592 0.13388875126838684 3.0000000000000004e-07 1.5000000000000002e-07
SELECT division_title FROM table_name_87 WHERE playoffs_appearances = "23" AND conference_titles = "1"</s>
-4.969192028045654
23 7144 -0.02076289989054203 0.07107175141572952 4.0000000000000003e-07 2.0000000000000002e-07
SELECT MAX(TOTAL) FROM table_name_61 WHERE gold < 1 AND bronze < 1 AND silver < 1 AND gold > 1 AND bronze > 1 AND silver > 1 AND gold > 1 AND bronze > 1 AND gold > 1 AND silver > 1 AND gold > 1 AND bronze > 1 AND gold > 1 AND silver > 1 AND gold > 1 AND bronze > 1 AND gold > 1 AND silver > 1 AND gold > 1 AND bronze > 1 AND gold > 1 AND silver > 1 AND gold > 1 AND bronze > 1 AND gold > 1 AND silver > 1 AND gold > 1 AND bronze > 1 AND gold > 1 AND silver > 1 AND gold > 1 AND bron

335 7144 0.021731732413172722 0.019196977838873863 4.3e-06 2.15e-06
select BIRTH_DATE from TABLE_NAME_3 where WEIGHT__KG_ = "100" and WORLD_RANK = "2"</s>
5.755478858947754
343 7144 0.010347575880587101 0.016551362350583076 4.4e-06 2.2e-06
select 2009 from TABLE_NAME_71 where 2011 = "SF"</s>
4.730398178100586
351 7144 -0.006458021234720945 0.006772099528461695 4.5e-06 2.25e-06
select DECILE from TABLE_NAME_99 where AUTHORITY = "STATE AUTHORITY" and ROLL = "120"</s>
5.752123832702637
359 7144 -0.018256478011608124 0.012758556753396988 4.600000000000001e-06 2.3000000000000004e-06
select DATE from TABLE_NAME_49 where GAME > 69 and OPPONENT = "ST. LOUIS BLUES"</s>
5.8697614669799805
367 7144 -0.0244454313069582 0.007603432983160019 4.7e-06 2.35e-06
select TEAM from TABLE_23285805_5 where DATE = "DECEMBER 19"</s>
4.554546356201172
375 7144 0.032890770584344864 0.028268737718462944 4.800000000000001e-06 2.4000000000000003e-06
select MIN(silver) from TABLE_NAME_68 where GOLD < 0</s>
5.4117641

711 7144 0.006517982110381126 0.0014100588159635663 9e-06 4.5e-06
select MIN(GUEST) from TABLE_25691838_12 where PRODUCTION_CODE = "6152"</s>
5.6593546867370605
719 7144 -0.008777322247624397 0.0019637083169072866 9.100000000000001e-06 4.5500000000000005e-06
select MUSIC from TABLE_NAME_60 where RECORD_LABEL = "MOTIVATE BOX" and RECORD_WAVES = "MOTIVATE RECORD"</s>
5.037847518920898
727 7144 -0.0027400683611631393 0.0008685711654834449 9.200000000000002e-06 4.600000000000001e-06
select DOWN__UP_TO_KBIT_S from TABLE_1773908_3 where PROVIDER = "DEUTSCHE TELEKOM AG"</s>
5.676542282104492
735 7144 0.015366323292255402 0.0020056741777807474 9.3e-06 4.65e-06
select AUTHOR from TABLE_NAME_16 where UNIT = "MALASUDA" and NAME = "VALENTINA"</s>
5.780348300933838
743 7144 0.004614296369254589 0.0006417194381356239 9.4e-06 4.7e-06
select METHOD from TABLE_NAME_19 where OPPONENT = "Thiago Alves"</s>
4.922232627868652
751 7144 -0.01691412553191185 0.002768877660855651 9.5e-06 4.75e-06
select SOCKET 

1071 7144 -0.011808217503130436 0.0010893353028222919 9.938441702975689e-06 4.9692208514878445e-06
select GOALS from TABLE_NAME_84 where ASSISTS = "33" and GAMES = "85"</s>
5.852644443511963
1079 7144 -0.005128490272909403 0.0010268338955938816 9.934881598487478e-06 4.967440799243739e-06
select HOME_GAMES from TABLE_28884858_1 where AVERAGE_ATTENDANCE = "79475"</s>
5.641493797302246
1087 7144 0.01987536996603012 0.0021973333787173033 9.931222095535204e-06 4.965611047767602e-06
select MIN(TOTAL) from TABLE_NAME_78 where NATION = "canada (can)" and BRONZE < 0</s>
6.014262676239014
1095 7144 0.01893678866326809 0.0025302923750132322 9.927463267828635e-06 4.963731633914317e-06
select ENGLISH from TABLE_26614365_1 where EAST as "light"</s>
5.618564605712891
1103 7144 0.009567228145897388 0.0009888582862913609 9.923605191078134e-06 4.961802595539067e-06
select CITY from TABLE_NAME_41 where ERP__ANALOG_DIGITAL = "600kw 500kw"</s>
5.127023696899414
1111 7144 -0.020534832030534744 0.00220584031

1415 7144 0.022002296522259712 0.0027758278883993626 9.696755231044618e-06 4.848377615522309e-06
select DATE from TABLE_NAME_69 where TOURNAMENT = "MILAN, itALY"</s>
5.360114097595215
1423 7144 0.010338093154132366 0.0009449781500734389 9.689012015199147e-06 4.844506007599573e-06
select MALE_LIFE_EXPERIENCE from TABLE_2701625_1 where COUNTRY = "pakistan"</s>
4.851866245269775
1431 7144 0.009558621793985367 0.005244977306574583 9.681174353198687e-06 4.8405871765993435e-06
select COMPOSER_S from TABLE_NAME_85 where LENGTH = "6:24"</s>
5.882352828979492
1439 7144 -0.04882045462727547 0.010995034128427505 9.673242402909555e-06 4.836621201454778e-06
select INSTITUTION from TABLE_NAME_78 where LOCATION = "AKRON"</s>
5.936544895172119
1447 7144 -0.06635109335184097 0.020015275105834007 9.665216324097222e-06 4.832608162048611e-06
select PLACE from TABLE_NAME_6 where MONEY___$__ = "73" and PLAYER = "ARCHARIC COMPSTON"</s>
5.788747310638428
1455 7144 -0.06763730198144913 0.02119455300271511 9.65

1767 7144 -0.015607263892889023 0.0016024785581976175 9.269052503341737e-06 4.634526251670869e-06
select COUNT(mountain_id) from mountains where camera_lens_id = "ETHIOPIA"</s>
5.312885284423828
1775 7144 -0.004942224361002445 0.0003870948567055166 9.257327672807383e-06 4.628663836403692e-06
select BILLS_POINTS from TABLE_NAME_54 where DATE = "NOVEMBER 11"</s>
6.130627632141113
1783 7144 0.01442668680101633 0.001323877600952983 9.24551709111097e-06 4.622758545555485e-06
select DATE from TABLE_NAME_66 where RESULT = "W20-0"</s>
5.443122386932373
1791 7144 0.028410406783223152 0.003852252848446369 9.233620996141421e-06 4.6168104980707105e-06
select CLASS from TABLE_NAME_6 where QUANTITY = "1"</s>
5.628842830657959
1799 7144 0.012749232351779938 0.0008872277685441077 9.221639627510076e-06 4.610819813755038e-06
select HOME from TABLE_NAME_31 where RECORD = "3-2"</s>
5.485411643981934
1807 7144 -0.0044966209679841995 0.000602404703386128 9.209573226545852e-06 4.604786613272926e-06
select co

2119 7144 0.010231131687760353 0.000676245370414108 8.675418298600884e-06 4.337709149300442e-06
select OBAMA_NUMBER from TABLE_20799905_1 where COUNTY = "LAKE COUNTY"</s>
5.362421035766602
2127 7144 -0.008723954670131207 0.0007445139926858246 8.660167579430926e-06 4.330083789715463e-06
select RACE from TABLE_NAME_3 where DATE = "10/10/2008"</s>
5.42064905166626
2135 7144 -0.016770772635936737 0.0015445343451574445 8.644843137107058e-06 4.322421568553529e-06
select AWAY_TEAM from TABLE_NAME_21 where CROWD > 6,000</s>
6.053621292114258
2143 7144 -0.015165111981332302 0.0013569002039730549 8.629445280294445e-06 4.314722640147222e-06
select DEPARTURE from TABLE_NAME_66 where STATION_NAME = "Rajendranagar Terminal"</s>
5.984164714813232
2151 7144 -0.011036969721317291 0.0012958166189491749 8.613974319136959e-06 4.3069871595684795e-06
select MANNER_OF_DEPARTURE from TABLE_27495117_3 where DATE_OF_APPOINTMENT = "21 september 2010"</s>
5.1301774978637695
2159 7144 0.014538797549903393 0.001426

2471 7144 0.009253652766346931 0.0009804638102650642 7.938926261462366e-06 3.969463130731183e-06
select cardinalatial_title from TABLE_NAME_40 where elector = "luccan" and elevated = "december 18, 1182"</s>
4.73054313659668
2479 7144 -0.0017807321855798364 0.00042110608774237335 7.920742425555436e-06 3.960371212777718e-06
select max(population__2002_census_data_) from TABLE_22854436_1 where 2002 census_data = "population__2002_census_data_"</s>
4.464055061340332
2487 7144 -0.008563969284296036 0.0006649757269769907 7.902499760007867e-06 3.951249880003934e-06
select POSITION from TABLE_11803648_20 where OVERALL = "126"</s>
5.25340461730957
2495 7144 0.0031870061066001654 0.0003407715994399041 7.884198632263725e-06 3.942099316131862e-06
select RESTYPE DESCRIPTION from TABLE_NAME where RESTYPE NAME = "SANDWICH"</s>
5.651818752288818
